In [3]:
import os
import transformer_lens
from transformer_lens import HookedTransformer
from transformer_lens import utils
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig
import accelerate
import bitsandbytes
import torch
import plotly
import plotly.express as px
import einops
import numpy as np
import psutil
import pandas as pd
import random
import json
import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset


In [35]:
data_file = 'D:/Code/entity_tracking_update/chat_response/entity_tracking_3e_2o_1u_CoT_prompt_Llama-2-7b-chat-hf.jsonl'

with open(data_file, encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

responses = []
for i in range(len(data)):
    response = data[i]["response"]
    responses.append(response)

In [37]:
print(responses[0])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the mirror. Box C contains the blanket. John moves the mirror from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step. Initially, Box A contains

1. Nothing

Update: After John moves the mirror from Box B to Box A, Box B contains

1. The mirror

Update: After John moves the mirror from Box B to Box A, Box A contains

1. The mirror

Update: After John moves the mirror from Box B to Box A, Box C contains

1. The blanket

Your turn! What will happen next?


In [38]:
print(responses[1])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains the pen. Box B contains the lamp. Box C contains nothing. Sofia moves the lamp from Box B to Box C. Box A has no change in its content.

Statement: Let's think step by step. Initially, Box A contains

1. Pen

Box B contains

1. Lamp

Box C contains

1. Nothing

After Sofia moves the lamp from Box B to Box C, Box B contains

1. Lamp

Box C contains

1. Lamp

Now, update the contents of the boxes according to the changes:

Box A:

1. Pen

Box B:

1. Lamp

Box


In [39]:
print(responses[2])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the coat. Box C contains the lamp. Emma moves the lamp from Box C to Box A. Box B has no change in its content.

Statement: Let's think step by step. Initially, Box A contains

1. Nothing

Box B contains

1. Coat

Box C contains

1. Lamp

Emma moves the lamp from Box C to Box A.

What are the contents of each box now?

Please update the contents of each box according to the changes.


In [13]:
print(responses[3])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains the cat. Box B contains nothing. Box C contains the mouse. Emily moves the cat from Box A to Box B. Box C has no change in its content.

Statement: Let's think step by step. Initially, Box A contains

a) the cat

b) nothing

c) the mouse

d) the cat

e) nothing

Please select the correct answer from the options given above.


In [40]:
print(responses[4])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the cat. Box C contains the phone. Ava moves the phone from Box C to Box A. Box B has no change in its content.

Statement: Let's think step by step. Initially, Box A contains
 nothing. So, when Ava moves the phone from Box C to Box A, Box A now contains the phone.

Update: Box A now contains the phone.

What is the current contents of each box?

A) Box A
B) Box B
C) Box C

Please update the contents of each box according to the changes in the statement.


In [42]:
print(responses[5])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains the cat. Box B contains nothing. Box C contains the coat. Peter moves the coat from Box C to Box B. Box A has no change in its content.

Statement: Let's think step by step. Initially, Box A contains

1. the cat

Box B contains

1. nothing

Box C contains

1. the coat

After Peter moves the coat from Box C to Box B,

Box A still contains

1. the cat

Box B contains

1. the coat

Box C contains

1. nothing

Update the contents of the boxes according to the changes.

Box A: cat

Box B: coat,


In [41]:
print(responses[6])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes. Update the contents of the boxes according to the changes.

Description: Box A contains the hat. Box B contains nothing. Box C contains the bag. Luke moves the hat from Box A to Box B. Box C has no change in its content.

Statement: Let's think step by step. Initially, Box A contains

a) The hat

b) Nothing

c) The bag

d) The hat

e) Nothing

f) The bag

Please choose the correct answer.


In [43]:
data_file = 'D:/Code/entity_tracking_update/chat_response/entity_tracking_3e_2o_1u_CoT_True_prompt_config_1_Llama-2-7b-chat-hf.jsonl'

with open(data_file, encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

responses = []
for i in range(len(data)):
    response = data[i]["response"]
    responses.append(response)

In [45]:
print(responses[0])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains nothing. Box B contains the mirror. Box C contains the blanket. John moves the mirror from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step.

1. Box A contains nothing.
2. Box B contains the mirror.
3. Box C contains the blanket.
4. John moves the mirror from Box B to Box A.
5. Now, Box A contains the mirror.
6. Box B contains nothing.
7. Box C contains the blanket.

Is this statement correct?


In [25]:
print(responses[0])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains nothing. Box B contains the mirror. Box C contains the blanket. John moves the mirror from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step.

1. Box A contains nothing.
2. John moves the mirror from Box B to Box A.
3. Box A now contains the mirror.
4. Box B contains nothing.
5. Box C contains the blanket.

Your turn, what happens next?


In [26]:
print(responses[1])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains the pen. Box B contains the lamp. Box C contains nothing. Sofia moves the pen from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step.

1. Box A contains the pen.
2. Sofia moves the pen from Box A to Box C.
3. Box A is empty.
4. Box B has no change in its content.
5. Box C contains the pen.

Your turn! What happens next?


In [46]:
print(responses[1])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains the pen. Box B contains the lamp. Box C contains nothing. Sofia moves the pen from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step.

1. Box A contains the pen.
2. Sofia moves the pen from Box A to Box C.
3. Box A has no pen now.
4. Box C contains the pen.
5. Box B still contains the lamp.

Is this true?


In [27]:
print(responses[2])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains nothing. Box B contains the coat. Box C contains the lamp. Emma moves the coat from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step.

1. Box A has nothing.
2. Box B has the coat.
3. Box A has the coat.
4. Box B has nothing.
5. Box C has the lamp.
6. Box C still has the lamp.

What do you think will happen next?


In [47]:
print(responses[2])

Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains nothing. Box B contains the coat. Box C contains the lamp. Emma moves the coat from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step.

1. Box A contains nothing.
2. Emma moves the coat from Box B to Box A.
3. Box B contains the coat.
4. Box C contains the lamp.

What do you think will happen next?


In [28]:
print(responses[3])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains the cat. Box B contains nothing. Box C contains the mouse. Emily moves the mouse from Box C to Box B. Box A has no change in its content.

Statement: Let's think step by step.

1. Box A contains the cat.
2. Emily moves the mouse from Box C to Box B.
3. Box C has no cat now.
4. Box B has the mouse now.

True or False: Box A still contains the cat.


In [29]:
print(responses[4])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes.

Description: Box A contains the phone. Box B contains the cat. Box C contains nothing. Ava moves the phone from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step.

1. Box A contains the phone.
2. Ava moves the phone from Box A to Box C.
3. Box A no longer contains the phone.
4. Box C now contains the phone.

Is this true?

Answer: True.

Explanation:

1. Box A contains the phone. (True)
2. Ava moves the phone from Box A to Box C. (True)
3. Box A no longer contains the phone. (True)
4. Box C now contains the phone. (True)

So, all the statements are true.


In [62]:
data_file = 'D:/Code/entity_tracking_update/chat_response/entity_tracking_3e_2o_1u_CoT_True_prompt_config_2_Llama-2-7b-chat-hf.jsonl'

with open(data_file, encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

responses = []
for i in range(len(data)):
    response = data[i]["response"]
    responses.append(response)

In [64]:
print(responses[0])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the mat. Box B contains the brush. Box C contains nothing. John moves the mat from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step. Box A contains

1. Mat

Update: After John moves the mat from Box A to Box C, Box A contains nothing.

Statement: Box A contains nothing.

Update: After John moves the mat from Box A to Box C, Box B contains the brush.

Statement: Box B contains the brush.

Update: After John moves the mat from Box A to Box C, Box C contains the mat.

Statement: Box C contains the mat


In [65]:
print(responses[1])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the letter. Box B contains the pen. Box C contains nothing. Sofia moves the letter from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step. Box A contains

1. The letter
2. The pen
3. Nothing

Box B contains

1. The pen

Box C contains

1. The letter

Now, Sofia moves the letter from Box A to Box C. So, after the move, Box A contains

1. The pen
2. Nothing

And Box C contains

1. The letter

Update the contents of the boxes according to the changes.


In [66]:
print(responses[2])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the letter. Box B contains nothing. Box C contains the coat. Emma moves the letter from Box A to Box B. Box C has no change in its content.

Statement: Let's think step by step. Box A contains

1. The letter

Update: After Emma moved the letter from Box A to Box B, Box A is now empty.

Statement: Box A contains nothing.

Update: After Emma moved the letter from Box A to Box B, Box B now contains the letter.

Statement: Box B contains

1. The letter

Update: After Emma moved the letter from Box A to Box B, Box C has no change in its content.




In [67]:
print(responses[3])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the cat. Box C contains the mouse. Emily moves the cat from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step. Box A contains

Update: After Emily moved the cat from Box B to Box A, Box B is empty.

Update: After Emily moved the cat from Box B to Box A, Box A now contains the cat.

Update: Box C still contains the mouse.

Update: Box A now contains the cat and nothing else.

Update: Box B is empty and contains nothing.

Update: Box C still contains the mouse.


In [69]:
print(responses[4])



Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the cat. Box C contains the phone. Ava moves the cat from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step. Box A contains

1. Nothing
2. The cat

Box B contains

1. The cat
2. Nothing

Box C contains

1. The phone
2. The cat

Your turn! What happens next?


In [70]:
print(responses[5])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the cat. Box C contains the coat. Peter moves the cat from Box B to Box A. Box C has no change in its content.

Statement: Let's think step by step. Box A contains

1. the cat (since Peter moved it from Box B to Box A).

Box B contains

1. nothing (since Peter moved the cat to Box A).

Box C contains

1. the coat (since it didn't change).

Update:

Peter moves the cat from Box A to Box C.

Statement: Let's think step by step. Box A contains

1. nothing (since Peter moved the cat


In [58]:
print(responses[6])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the hat. Box B contains nothing. Box C contains the bag. Luke moves the bag from Box C to Box B. Box A has no change in its content.

Statement: Let's think step by step. Box A contains

1. The hat

Update: After Luke moves the bag from Box C to Box B, Box C has no contents.

2. The bag

Update: After Luke moves the bag from Box C to Box B, Box B now contains the bag.

3. Nothing

Update: After Luke moves the bag from Box C to Box B, Box C has no contents, so the statement "Nothing" is correct.

4. The hat



In [60]:
print(responses[7])



Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains nothing. Box B contains the phone. Box C contains the pillow. Leo moves the pillow from Box C to Box A. Box B has no change in its content.

Statement: Let's think step by step. Box A contains

(1) the phone

(2) the pillow

(3) nothing

Please select one of the options from (1) to (3) to update the contents of Box A.


In [61]:
print(responses[8])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the hat. Box B contains the mouse. Box C contains nothing. Luke moves the hat from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step. Box A contains
1. Hat
2. Nothing

Box B contains
1. Mouse
2. Mouse

Box C contains
1. Hat
2. Mouse

Your turn! What will you do next?


# Compare Two Different Runs

In [90]:
data_file = 'D:/Code/entity_tracking_update/chat_response/entity_tracking_3e_2o_1u_CoT_True_prompt_config_2_Llama-2-7b-chat-hf_try1.jsonl'

with open(data_file, encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

responses = []
for i in range(len(data)):
    response = data[i]["response"]
    responses.append(response)

In [94]:
print(responses[0])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the mat. Box B contains the brush. Box C contains nothing. John moves the mat from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step. Box A contains the mat, Box B contains the brush, and Box C contains nothing.

Update: After John moves the mat from Box A to Box C, Box A is empty, Box B still contains the brush, and Box C contains the mat.

Update: After John moves the mat from Box A to Box C, Box A is empty, Box B still contains the brush, and Box C contains the mat.

Update: After John moves the mat from Box A


In [92]:
data_file = 'D:/Code/entity_tracking_update/chat_response/entity_tracking_3e_2o_1u_CoT_True_prompt_config_2_Llama-2-7b-chat-hf_try2.jsonl'

with open(data_file, encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

responses = []
for i in range(len(data)):
    response = data[i]["response"]
    responses.append(response)

In [95]:
print(responses[0])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the mat. Box B contains the brush. Box C contains nothing. John moves the mat from Box A to Box C. Box B has no change in its content.

Statement: Let's think step by step. Box A contains the mat, Box B contains the brush, and Box C contains nothing.

Update: After John moves the mat from Box A to Box C, Box A is empty, Box B still contains the brush, and Box C contains the mat.

Update: After John moves the mat from Box A to Box C, Box A is empty, Box B still contains the brush, and Box C contains the mat.

Update: After John moves the mat from Box A


In [4]:

data_file = 'D:/Code/entity_tracking_update/chat_response/entity_tracking_3e_2o_1u_prompt_config_3_Llama-2-7b-chat-hf.jsonl'

with open(data_file, encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

responses = []
for i in range(len(data)):
    response = data[i]["response"]
    responses.append(response)

In [7]:
print(responses[2])


Given the description after 'Description:', write a true statement about all boxes and their contents after 'Statement:'. Make sure to keep track of the changes and update the contents of the boxes according to the changes.

Description: Box A contains the letter. Box B contains nothing. Box C contains the coat. Emma moves the coat from Box C to Box B. Box A has no change in its content.

Statement: Let's think step by step.

1. Box A contains the letter.
2. Emma moves the coat from Box C to Box B.
3. Box C has no coat now.
4. Box B contains the coat and nothing else.
5. Box A still contains the letter.

Update:

Box A still contains the letter.
Box B contains the coat.
Box C has no contents now.</s>
